# Bounds on the Joint Probability for Rayleigh Fading Scenarios

_Author:_ Karl-Ludwig Besser, Technische Universität Braunschweig, Germany

This notebook is part of the paper "Copula-Based Bounds for Multi-User Communications - Part II: Outage Performance" ([doi:10.1109/LCOMM.2020.3023050](https://doi.org/10.1109/LCOMM.2020.3023050)).  
If you use any of this work, please cite the above paper.

> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. The will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [1]:
import numpy as np
from scipy import stats
from scipy import optimize
from scipy import special
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact

# Scenario

We consider a channel with fading and interference: Alice wants to transmit a message to Bob over a channel with gain $X$. At the same time, another user transmits which causes interference modeled by $Y$.

In the following, we will consider the case of Rayleigh fading. The receive signals $X$ and $Y$ are therefore exponentially distributed, i.e., $X\sim\exp(\lambda_{x})$, $Y\sim\exp(\lambda_{y})$.

# Product of Random Variables

In the following, we show the bounds on the outage probability of a RIS-like channel, i.e.,
$$ \Pr\left(\log_2\left(1+XY\right)<R\right). $$

In [2]:
def cdf_exp(x, lam=1):
    return 1-np.exp(-lam*x)

## Lower Bound

In [3]:
def g_lower(y, s, lam_x, lam_y):
    idx_nonzero = np.where(y != 0)
    results = np.zeros(np.shape(y))
    results[idx_nonzero] = cdf_exp(s/y[idx_nonzero], lam=lam_x) + cdf_exp(y[idx_nonzero], lam=lam_y) - 1
    return results
    #return cdf_exp(s/y, lam=lam_x) + cdf_exp(y, lam=lam_y) - 1

def deriv_g_lower(y, s, lam_x, lam_y):
    return lam_y*np.exp(-lam_y*y) - lam_x*s/y**2*np.exp(-lam_x*s/y)

In [4]:
def negative_g_lower_scalar(y, s, lam_x, lam_y):
    if y == 0:
        return 0
    else:
        return -(cdf_exp(s/y, lam=lam_x) + cdf_exp(y, lam=lam_y) - 1)

def plot_g_lower():
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    y = np.linspace(0.001, 5)
    ax2.plot([0, 5], [0, 0], color='gray', alpha=.2)
    plots = {"g": ax1.plot(y, np.zeros(len(y)), 'b', label="g")[0],
             "deriv_g": ax2.plot(y, np.zeros(len(y)), 'r', label="g'")[0],
             "gopt": ax1.plot([0, 5], [0, 0], 'g', label="max(g)")[0],
             "yopt": ax1.plot([0, 0], [0, 1], 'g--', label="y_opt")[0]
            }
    ax1.set_xlim([0, 5])
    ax1.set_ylim([0, 1])
    ax1.set_xlabel("y")
    ax1.set_ylabel("g(y)")
    ax2.set_ylim([-.5, .5])
    ax2.set_ylabel("g'")
    fig.legend()
    def update_plot(rate, lam_x, lam_y):
        s = 2**rate - 1
        maximum = optimize.minimize_scalar(negative_g_lower_scalar, args=(s, lam_x, lam_y))
        print(maximum)
        plots["g"].set_ydata(g_lower(y, s, lam_x, lam_y))
        plots["deriv_g"].set_ydata(deriv_g_lower(y, s, lam_x, lam_y))
        plots["gopt"].set_ydata([-maximum.fun]*2)
        plots["yopt"].set_xdata([maximum.x]*2)
    interact(update_plot, rate=(0.1, 2, .1), lam_x=(.1, 3, .1), lam_y=(.1, 3, .1))

In [5]:
plot_g_lower()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='rate', max=2.0, min=0.1), FloatSlider(value=1.500000…

## Upper Bound

In [6]:
def g_upper(y, s, lam_x, lam_y):
    idx_nonzero = np.where(y != 0)
    results = np.zeros(np.shape(y))
    results[idx_nonzero] = cdf_exp(s/y[idx_nonzero], lam=lam_x) + cdf_exp(y[idx_nonzero], lam=lam_y)
    return np.minimum(results, 1)

def g_upper_scalar(y, s, lam_x, lam_y):
    if y == 0:
        return 1
    else:
        return np.minimum(cdf_exp(s/y, lam=lam_x) + cdf_exp(y, lam=lam_y), 1)

def plot_g_upper():
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    y = np.linspace(0.001, 5)
    ax2.plot([0, 5], [0, 0], color='gray', alpha=.2)
    plots = {"g": ax1.plot(y, np.zeros(len(y)), 'b', label="g")[0],
             "deriv_g": ax2.plot(y, np.zeros(len(y)), 'r', label="g'")[0],
             "gopt": ax1.plot([0, 5], [0, 0], 'g', label="min(g)")[0],
             "yopt": ax1.plot([0, 0], [0, 1], 'g--', label="y_opt")[0]
            }
    ax1.set_xlim([0, 5])
    ax1.set_ylim([0, 1.05])
    ax1.set_xlabel("y")
    ax1.set_ylabel("g(y)")
    ax2.set_ylim([-.5, .5])
    ax2.set_ylabel("g'")
    fig.legend()
    def update_plot(rate, lam_x, lam_y):
        s = 2**rate - 1
        maximum = optimize.minimize_scalar(g_upper_scalar, args=(s, lam_x, lam_y))
        print(maximum)
        plots["g"].set_ydata(g_upper(y, s, lam_x, lam_y))
        plots["deriv_g"].set_ydata(deriv_g_lower(y, s, lam_x, lam_y))
        plots["gopt"].set_ydata([maximum.fun]*2)
        plots["yopt"].set_xdata([maximum.x]*2)
    interact(update_plot, rate=(0.1, 1, .1), lam_x=(.1, 1, .1), lam_y=(.1, 1, .1))

In [7]:
plot_g_upper()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, description='rate', max=1.0, min=0.1), FloatSlider(value=0.5, des…

## Summary

In [8]:
@np.vectorize
def lower_bound(s, lam_x, lam_y):
    maximum = optimize.minimize_scalar(negative_g_lower_scalar, args=(s, lam_x, lam_y))
    return -maximum.fun

@np.vectorize
def upper_bound(s, lam_x, lam_y):
    minimum = optimize.minimize_scalar(g_upper_scalar, args=(s, lam_x, lam_y))
    return minimum.fun

def indep_channels(s, lam_x, lam_y):
    _a = 2 * np.sqrt(lam_x*lam_y*s)
    return 1 - _a * special.kn(1, _a)

def plot_bounds_vs_rate():
    fig, ax1 = plt.subplots(num="Outage Probability vs Rate")
    rate = np.logspace(-3, 1, 50)
    s = 2**rate - 1
    curves = {"Lower Bound": lower_bound, "Upper Bound": upper_bound, "Independent Channels": indep_channels}
    plots = {k: ax1.loglog(rate, np.ones(len(rate)), label=k)[0] for k in curves.keys()}
    ax1.set_xlim([1e-3, 1e1])
    ax1.set_ylim([1e-5, 1.1])
    ax1.set_xlabel("Rate $R$")
    ax1.set_ylabel("$\\varepsilon$")
    fig.legend()
    def update_plot(lam_x, lam_y):
        for _name, _func in curves.items():
            plots[_name].set_ydata(_func(s, lam_x, lam_y))
    interact(update_plot, lam_x=(.5, 3, .1), lam_y=(.5, 3, .1))

In [9]:
plot_bounds_vs_rate()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.7000000000000002, description='lam_x', max=3.0, min=0.5), FloatSlide…

In [10]:
def export_results(lam_x=1, lam_y=1):
    import pandas as pd
    rate = np.logspace(-3, 1, 50)
    s = 2**rate - 1
    curves = {"lower": lower_bound, "upper": upper_bound, "indep": indep_channels}
    results = {"rate": rate}
    for _name, _func in curves.items():
        results[_name] = _func(s, lam_x, lam_y)
    filename = "bounds-product-rayleigh-lx{}-ly{}.dat".format(lam_x, lam_y)
    data = pd.DataFrame.from_dict(results)
    data.to_csv(filename, sep='\t', index=False)

export_results()